# İçerik Temelli Filtreleme ( Content Based Filtering )

Ürün içeriklerinin benzerlikleri üzerinden tavsiyeler geliştirir. Örneğin bir filmin, kitabın açıklaması, bir filmin oyuncu kadrosu üzerinden benzerlikler hesaplanır ve başka bir tavsiye önerilir.  

Örneğin bir ürün satın alındığında, satın alınan ürünün açıklamasına en benzer açıklamaya sahip olan ürünlerin tavsiye edilmesi yöntemidir.

Örnek olarak bir kullanıcı bir şarkı dinlediğinde sahip olduğumuz tek bilgi, dinlediği şarkıdır. Bu kullanıcıya dinlediği şarkının türünde başka bir şarkı önermek, içerik temelli filtreleme sonucu bir öneride bulunmaya örnektir.

1. Metinlerin matematiksel olarak temsil edilmesi
2. Benzerlik hesaplama 

- Yukarıdaki bilgilerden yola çıkarak bizim iki meta bilgi (açıklama, metin ifadesi) arasındaki uzaklığı bakmamız gerekir.
- Peki, İki metin arasındaki uzaklık nasıl hesaplanır?

- Bu soruna çözüm bulabilmek için metinleri vektörleştireceğiz. 
- Metinlerin vektörleştirilmesi metinlerin matematiksel olarak ifade edilmesine dayanır.

## Metinlerin vektörel temsilleri
Temeli metinlerin matematiksel olarak ölçülebilir bir forma dönüştürülermektir. 

Örneğin;

Açıklamalardaki kelimeler film indeksinde vektörleştirilerek matris oluşturulur. Bu kelimeler özelinde kullanım sayılarına göre sayısal değerlerle matrisimizde betimlenir.

Metinleri vektörlerle temsil etmenin iki yolu vardır. Bunlar;
1. Count vector (word count, sayım vektörü): 
- Metinsel ifadeleri matematiksel metriklere dönüştürme tekniğidir. 
- Öncelikle eşsiz tüm terimleri (kelimeleri) sütunlara, bütün dokümanları (satırlara konulacak kavramlar, film açıklamaları, tweetler, ürün başlıkları vb.) satırlara yerleştiririz. 
- Ardından terimlerin dokümanlarda geçme frekansları hücrelere yerleştirilir. Oluşan tablomuz vektörleştirilmiş bir matris halinde olur.
- Öklid uzaklığı hesabı ve Cosine Similarity hesabı yapılabilir hale gelir.




2. TF-IDF (term frequency-inverse document frequency): 
- Kelimelerin hem kendi metinlerinde, hem de bütün odaklanılan verideki geçme frekansları üzerinden bir normalizasyon işlemi yapar. 
- Count vector yönteminde ortaya çıkabilecek bazı yanlılıkları (frekansı yüksek olan değerlerin analiz sonuçlarını yanıltması gibi) giderir.

## TF-IDF Hesaplanması

TF-IDF beş adımda hesaplanabilir:

1. Adım: Count vektörü hesapla. Count vector yani kelimelerin her bir dokümandaki frekansı hesaplanır. 

2. Adım: Term frequency hesapla. Yani terimlerin frekansları hesaplanır. 

        TF = t teriminin ilgili dökümandaki frekansı / dokümandaki toplam terim sayısı

3. Adım: Inverse document frequency (IDF) hesaplanması.

        IDF=1+(loge((toplam doküman sayısı+1)/içinde t terimi olan doküman sayısı+1)) 


4. Adım: TF*IDF çarpılarak hesaplanır.

5. Adım: L2 Normalizasyonu yapılır. 

        Satırların kareleri toplamının karekökü bulunur. 
        İlgili satırdaki tüm hücreler, bulunan değere bölünür. 


Yukarıdaki işlemler incelendiğinde;

- Term Frequency'de (TF) dökümanların kendi içerisindeki dağılımları ile ilgilendik.
- Inverse Document Frequency'de (IDF) ise bütün dökümalar açısından kelime dağılımları incelenmiş oldu. 
- Bu ik skoru çarparak ortak bir puan hesaplanmış oluyor.
- L2 normalizasyonu ise; bir meta bilginin içerisindeki bütün kelimelerin etkileri işlenmiştir.

## Film Overview'larına Göre Tavsiye Geliştirme


- Verisetinde yaklaşık 45bin kadar film bulunmaktadır. 
- Uygulamamız kapsamında filmlerin açıklamalarını içeren overwiev değişkeni ile çalışacağız. 

1. TF-IDF Matrisinin Oluşturulması
2. Cosine Similarity Matrisinin Oluşturulması
3. Benzerliklere Göre Önerilerin Yapılması

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# https://www.kaggle.com/rounakbanik/the-movies-dataset
df = pd.read_csv("datasets/the_movies_dataset/movies_metadata.csv", low_memory=False)  # DtypeWarning kapamak icin
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
df.shape

(45466, 24)

In [6]:
df["overview"].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [7]:
tfidf= TfidfVectorizer(stop_words="english")
# "stop_words=" argümanı ölçüm değeri taşımayan ifadeleri metinden çıkarır. "the", "on" vs.

In [8]:
df[df["overview"].isnull()] 
# overwiev bölümü Na olanlar geldi. Bu sebeple bu verileri çalışmanın dışında bulunabilir ya da boşluklarla değiştirebiliriz. 

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
32,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...",NaN,78802,tt0114952,fr,"Guillaumet, les ailes du courage",NaN,...,1996-09-18,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wings of Courage,False,6.8,4.0
300,False,NaN,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,161495,tt0114296,sv,Roommates,NaN,...,1995-03-01,12400000.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Roommates,False,6.4,7.0
634,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,287305,tt0117312,de,Peanuts – Die Bank zahlt alles,NaN,...,1996-03-21,0.0,NaN,[],Released,NaN,Peanuts – Die Bank zahlt alles,False,4.0,1.0
635,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,339428,tt0116485,de,Happy Weekend,NaN,...,1996-03-14,65335.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Happy Weekend,False,0.0,0.0
641,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,10801,tt0117788,de,Das Superweib,NaN,...,1996-03-06,0.0,86.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,The Superwife,False,5.3,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45342,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,199887,tt1771636,en,Over/Under,NaN,...,2013-01-04,0.0,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Over/Under,False,4.0,2.0
45377,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}]",NaN,317389,tt0070695,es,Simbad e il califfo di Bagdad,NaN,...,1973-07-22,0.0,NaN,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Simbad e il califfo di Bagdad,False,0.0,0.0
45398,False,NaN,1254040,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://lmtr.fi/,468707,tt5742932,fi,Lauri Mäntyvaaran tuuheet ripset,NaN,...,2017-07-28,0.0,90.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,NaN,Thick Lashes of Lauri Mäntyvaara,False,8.0,1.0
45399,False,NaN,750000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,280422,tt3805180,ru,Все и сразу,NaN,...,2014-06-05,3.0,0.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,All at Once,False,6.0,4.0


In [9]:
df["overview"] =df["overview"].fillna('')
#Na değerlerini boşluklarla değiştirdik. 

In [10]:
# Şimdi ise oluşturduğumuz nesne ile bir matriks oluşturacağız.

tfidf_matrix = tfidf.fit_transform(df["overview"])

# tfidf nesnesi ile bir matriks oluşturup buna fit edip sonrasında ise dönüştürme işlemi gerçekleştirilir.
# bakalım bunun içerisinde kaç eleman var

In [11]:
tfidf_matrix.shape
# Aşağıdaki ifadede satırlardakiler, film, yorum, açıklama, yorumlardır... Stünlardakiler eşsiz kelimeleri ifade eder. 


(45466, 75827)

In [12]:
tfidf.get_feature_names()

# ".get_feature_names()" metodu TfidfVectorizer() fonksiyonu ile olşturulan nesneye özel bir metoddur.
# Yaptığımız örnekte 75827 farklı kelimeyi gösterir bize. 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '000km',
 '000th',
 '001',
 '006',
 '007',
 '008',
 '009',
 '0093',
 '01',
 '0123',
 '02',
 '03',
 '04',
 '042',
 '05',
 '05pm',
 '06',
 '07',
 '077',
 '07am',
 '08',
 '088',
 '09',
 '10',
 '100',
 '1000',
 '10000',
 '1000s',
 '1000th',
 '1001',
 '100th',
 '101',
 '101st',
 '103',
 '103rd',
 '104',
 '105',
 '1066',
 '108',
 '1080s',
 '108th',
 '109',
 '10b',
 '10crores',
 '10mn',
 '10th',
 '10x',
 '11',
 '110',
 '1100',
 '111',
 '112',
 '1138',
 '114',
 '115',
 '117',
 '117a',
 '118',
 '1183',
 '119',
 '11s',
 '11th',
 '12',
 '120',
 '1200',
 '1200s',
 '1206',
 '1215',
 '1218',
 '1227',
 '125',
 '1250',
 '125th',
 '1263',
 '129',
 '12th',
 '13',
 '130',
 '1300',
 '1300s',
 '1302',
 '1303',
 '133',
 '134',
 '1344',
 '1348',
 '1349',
 '138',
 '13anos',
 '13b',
 '13s',
 '13th',
 '14',
 '140',
 '1400',
 '1408',
 '1413',
 '142',
 '1429',
 '143',
 '144',
 '145',
 '1458',
 '146',
 '1463',
 '1466',
 '1472',
 '1475',
 '148',
 '1482',
 '1483',
 '1492',
 '14pm',
 '14th',
 '15',
 '

In [13]:
# kesişimleri ile ilgili bilgilere de erişmek istiyoruz bunun için. Arraye çevireceğiz. 
# matriks'i array e dönüştürmek istersek;

tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 2. Cosine Similarity Matrisinin Oluşturulması

- Elimizde metinler vardı ancak biz bunları matematiksel olarak ifade etmek istiyorduk. Bu sebeple öncelikle metin vektörlerini oluşturduk. Şimdi ise uzaklık/benzerlik temelli bazı yaklaşımlarla hangi ürün(filmlerin) benzer olduğunu bulmaya odaklanacağız. 
- Cosine smilarity gibi bir çok benzerlik ve uzaklık ölçüsü vardır. Bunlar; Manhattan Distance, Euclide Distance, Pearson Correlation. 
- Genel olarak  Cosine Similarity ve Euclide Distance ölçülerini kullanılmaktadır. 

In [14]:

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# elimizdeki bütün döküman çiftleri için cosine similarity hesaplaması yapılmaktadır. 

In [15]:
cosine_sim.shape 

(45466, 45466)